Proposed Research Questions:
1. What is/are the variables that can mostly affect the no. of installs of an app?
2. What characteristics of a paid app can help in increasing sales (no. of installs) of an app?
3. What is/are the variables that can mostly affect the rating of an app?
    - Are ratings affected by application pricing?
    - What characteristics of a paid app can help in improving the rating of an app?
    - Are ratings relevant in user interest (no. of installs) of an app?
4. Are genres associated with target audiences?
5. Is it possible to suggest the rating of an app given the variables?
